# Kimberly Home Valuations — ZIP-level (Zillow ZHVI)

This notebook builds **ZIP-level** home value trends (ZHVI) for:
- 35091 (Kimberly), 35116 (Morris), 35180 (Warrior).

**Notes**
- ZIP areas are not city boundaries; used here for local market trend context.
- Source: Zillow Research (ZHVI). Last run: Aug 21, 2025.


In [ ]:
# Bootstrap: folders and paths (safe to re-run)
from pathlib import Path
import shutil, pandas as pd, matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
DATA_RAW = ROOT / "data" / "raw"
DATA_PROCESSED = ROOT / "data" / "processed"
CHARTS = ROOT / "charts"
OUTPUTS = ROOT / "outputs"

for p in [DATA_RAW, DATA_PROCESSED, CHARTS, OUTPUTS]:
    p.mkdir(parents=True, exist_ok=True)

# Helper to stamp chart footer
def add_footer(ax, text):
    ax.figure.text(0.01, 0.01, text, ha="left", va="bottom", fontsize=9, alpha=0.8)

plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:

# Auto-find ZHVI CSVs in data/raw (prefer known filenames; fall back to any *zhvi*)
from pathlib import Path
import pandas as pd

def find_one(patterns):
    for pat in patterns:
        hits = sorted(DATA_RAW.glob(pat))
        if hits:
            # choose newest by modified time
            hits.sort(key=lambda p: p.stat().st_mtime, reverse=True)
            return hits[0]
    raise FileNotFoundError(f"No ZHVI file found in {DATA_RAW}. Expected patterns: {patterns}")

ZILLOW_VALUES_CSV = find_one([
    "*Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month*.csv",
    "*zhvi*zip*month*.csv",
    "*zhvi*.csv",
])

print("Using ZHVI file:", ZILLOW_VALUES_CSV.name)
values_wide = pd.read_csv(ZILLOW_VALUES_CSV)


In [ ]:

import pandas as pd

ZIPS_TO_PLOT = [35091, 35116, 35180]
START_YEAR, END_YEAR = 2000, 2025

def reshape_zip_values(df: pd.DataFrame, zip_code: int, start_year=START_YEAR, end_year=END_YEAR) -> pd.DataFrame:
    z = df[df["RegionName"] == zip_code].copy()
    if z.empty:
        raise ValueError(f"ZIP {zip_code} not found in ZHVI file.")
    meta_cols = ["RegionName", "City", "CountyName", "Metro", "StateName", "State"]
    drop_cols = ["RegionID", "SizeRank", "RegionType"]
    date_cols = [c for c in z.columns if c not in meta_cols + drop_cols]
    long_df = z.melt(id_vars=meta_cols, value_vars=date_cols, var_name="Date", value_name="MedianHomeValue")
    long_df["Date"] = pd.to_datetime(long_df["Date"], errors="coerce")
    long_df = long_df.dropna(subset=["Date", "MedianHomeValue"])
    long_df = long_df[(long_df["Date"].dt.year >= start_year) & (long_df["Date"].dt.year <= end_year)]
    city = z["City"].iloc[0] if pd.notna(z["City"].iloc[0]) else f"ZIP {zip_code}"
    long_df["SeriesLabel"] = f"{city} ({zip_code})"
    return long_df[["Date","MedianHomeValue","SeriesLabel","RegionName","City","CountyName","Metro","StateName","State"]].sort_values("Date")

def build_values_timeseries(df: pd.DataFrame, zips: list[int]) -> pd.DataFrame:
    frames = [reshape_zip_values(df, z) for z in zips]
    return pd.concat(frames, ignore_index=True)

def latest_snapshot(values_tidy: pd.DataFrame) -> pd.DataFrame:
    idx = values_tidy.groupby("SeriesLabel")["Date"].idxmax()
    snap = values_tidy.loc[idx, ["SeriesLabel","Date","MedianHomeValue"]].sort_values("SeriesLabel")
    snap["MedianHomeValue"] = snap["MedianHomeValue"].round(0).astype(int)
    return snap.reset_index(drop=True)

values_tidy = build_values_timeseries(values_wide, ZIPS_TO_PLOT)
latest = latest_snapshot(values_tidy)
display(latest)


In [ ]:

import matplotlib.pyplot as plt

plt.figure()
for label, grp in values_tidy.groupby("SeriesLabel"):
    plt.plot(grp["Date"], grp["MedianHomeValue"], linewidth=2, label=label)

plt.title("Median Home Value Trend (ZHVI) — 2000–2025")
plt.xlabel("Year"); plt.ylabel("Median Home Value ($)")
plt.grid(True, linestyle="--", alpha=0.6); plt.legend(); plt.tight_layout()

# Footer with latest month in data
latest_month = values_tidy["Date"].max().strftime("%b %Y")
add_footer(plt.gca(), f"Source: Zillow ZHVI • {latest_month} • ZIP-level (35091/35116/35180; ZIP ≠ city boundary)")

out_png = CHARTS / "home_values_35091_35116_35180_2000_2025.png"
plt.savefig(out_png, dpi=200); plt.show()
print("Saved chart:", out_png)

# Export processed snapshot
out_csv = DATA_PROCESSED / "zhvi_latest_snapshot.csv"
latest.to_csv(out_csv, index=False)
print("Saved:", out_csv)
